In [4]:
import pandas as pd
import numpy as np
# import data_sampling as ds
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn import svm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
import time
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.options.display.max_colwidth = 150

data_iris = pd.read_csv("iris.csv")
data_glass = pd.read_csv("glass.data")

# to store the models and their results (to report)
models, results, tempos = list(), list(), list()

print('Iris:', np.shape(data_iris))
print('Glass:', np.shape(data_glass))

Iris: (150, 5)
Glass: (214, 11)


In [5]:
data_glass = shuffle(data_glass)
data_glass.head()

,a,b,c,d,e,f,g,h,i,j,target
123,124,1.51707,13.48,3.48,1.71,72.52,0.62,7.99,0.00,0.00,2
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.00,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.00,1
11,12,1.51763,12.80,3.66,1.27,73.01,0.60,8.56,0.00,0.00,1
100,101,1.51655,12.75,2.85,1.44,73.27,0.57,8.79,0.11,0.22,2


In [6]:
data_iris = shuffle(data_iris)
data_iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
143,6.8,3.2,5.9,2.3,Virginica
73,6.1,2.8,4.7,1.2,Versicolor
134,6.1,2.6,5.6,1.4,Virginica
142,5.8,2.7,5.1,1.9,Virginica
80,5.5,2.4,3.8,1.1,Versicolor


In [7]:
y = data_glass['target']
data_glass = data_glass.drop(columns = ['target', 'a'])

In [8]:
y_iris = data_iris['variety']
data_iris = data_iris.drop(columns = ['variety'])

In [9]:
# Normalizing the Glass dataset: standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
scaler.fit(data_glass)
data_glass[:] = scaler.transform(data_glass)
data_glass.head()

,b,c,d,e,f,g,h,i,j
123,-0.427565,0.088560,0.552779,0.532207,-0.169443,0.188951,-0.681044,-0.352877,-0.586451
0,0.872868,0.284953,1.254639,-0.692442,-1.127082,-0.671705,-0.145766,-0.352877,-0.586451
2,-0.721318,0.149933,0.601422,0.190912,0.438787,-0.164533,-0.828949,-0.352877,-0.586451
11,-0.242732,-0.746110,0.677863,-0.351146,0.464669,0.158213,-0.279586,-0.352877,-0.586451
100,-0.599196,-0.807482,0.114985,-0.009850,0.801137,0.112107,-0.117594,-0.131128,1.676673


In [10]:
all_models=[MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(16, 8), activation='logistic', 
                            random_state=5), 
            Perceptron(tol=1e-3, random_state=5), 
            svm.SVC(gamma='scale', kernel='poly', decision_function_shape='ovr', random_state=5),
            svm.SVC(gamma='scale', kernel='rbf', decision_function_shape='ovr', random_state=5), 
            svm.SVC(gamma='scale', kernel='sigmoid', decision_function_shape='ovr', random_state=5),
            svm.SVC(gamma='scale', kernel='poly', decision_function_shape='ovo', random_state=5),
            svm.SVC(gamma='scale', kernel='rbf', decision_function_shape='ovo', random_state=5), 
            svm.SVC(gamma='scale', kernel='sigmoid', decision_function_shape='ovo', random_state=5)]

models.extend(all_models)

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=2019)    

Running models for Glass dataset:

In [12]:
#For each model:
for model in all_models:
    print('\n\n======= Model: ', model, '=======')
    # for each fold:
    accuracies = list()
    times = list()
    
    for i, (train_index, test_index) in enumerate(kf.split(data_glass)):
        #Builds the train and validation dataset, according to the current fold:
        y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
        X_train, X_valid = data_glass.iloc[train_index,:].copy(), data_glass.iloc[test_index,:].copy()
        # print("Fold", i)
        start = time.time()
        model.fit(X_train, y_train)
        end = time.time()
        times.append(end - start)
        pred = model.predict(X_valid)
        # print(confusion_matrix(y_valid, pred, labels=[1,2,3,5,6,7]))
        acc = accuracy_score(pred, y_valid)
        accuracies.append(acc)
        
    print('====\nFinal results: \nMean accuracy:', np.mean(accuracies))
    print('Mean traning model time: ', np.mean(times))
    results.append(np.mean(accuracies))
    tempos.append(np.mean(times))



======= Model:  MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(16, 8), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=5, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False) =======
====
Final results: 
Mean accuracy: 0.6449612403100775
Mean traning model time:  0.11502652168273926


======= Model:  Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=5, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False) =======
====
Final results: 
Mean accuracy: 0.5139534883720931
Mean tr

In [10]:
data = {'Accuracy': results, 'Time': tempos, 'Model': models}
dataframe = pd.DataFrame(data=data)
dataframe = dataframe.sort_values(by=['Accuracy'], ascending=False)
dataframe

,Accuracy,Model,Time
3,0.687043,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',\n max_iter=-1, ...",0.003691
6,0.687043,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',\n max_iter=-1, ...",0.006276
0,0.659136,"MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',\n beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,\n ...",0.223193
1,0.565227,"Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,\n fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_chang...",0.006509
4,0.541971,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',\n max_iter=...",0.006637
7,0.541971,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovo', degree=3, gamma='scale', kernel='sigmoid',\n max_iter=...",0.003629
2,0.490476,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',\n max_iter=-1,...",0.002806
5,0.490476,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovo', degree=3, gamma='scale', kernel='poly',\n max_iter=-1,...",0.007367


In [11]:
dataframe.to_csv('glass_results.csv',index=False)

In [12]:
# Normalizing the Iris dataset: standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
scaler.fit(data_iris)
data_iris[:] = scaler.transform(data_iris)
data_iris.head()

,sepal.length,sepal.width,petal.length,petal.width
84,-0.537178,-0.131979,0.421734,0.395774
69,-0.294842,-1.282963,0.080709,-0.130755
12,-1.264185,-0.131979,-1.340227,-1.447076
118,2.249683,-1.052767,1.785832,1.448832
139,1.280340,0.098217,0.933271,1.185567


In [13]:
models, results, tempos = list(), list(), list()

In [14]:
# Restarting the models (would them keep the parameters of last run?)
all_models=[MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(16, 8), activation='logistic', 
                            random_state=5), 
            Perceptron(tol=1e-3, random_state=5), 
            svm.SVC(gamma='scale', kernel='poly', decision_function_shape='ovr', random_state=5),
            svm.SVC(gamma='scale', kernel='rbf', decision_function_shape='ovr', random_state=5), 
            svm.SVC(gamma='scale', kernel='sigmoid', decision_function_shape='ovr', random_state=5),
            svm.SVC(gamma='scale', kernel='poly', decision_function_shape='ovo', random_state=5),
            svm.SVC(gamma='scale', kernel='rbf', decision_function_shape='ovo', random_state=5), 
            svm.SVC(gamma='scale', kernel='sigmoid', decision_function_shape='ovo', random_state=5)]
models.extend(all_models)

Running models for Iris dataset:

In [15]:
#For each model:
for model in all_models:
    print('\n\n======= Model: ', model, '=======')
    # for each fold:
    accuracies = list()
    times = list()
    
    for i, (train_index, test_index) in enumerate(kf.split(data_iris)):
        #Builds the train and validation dataset, according to the current fold:
        y_train, y_valid = y_iris.iloc[train_index].copy(), y_iris.iloc[test_index]
        X_train, X_valid = data_iris.iloc[train_index,:].copy(), data_iris.iloc[test_index,:].copy()
        # print("Fold", i)
        start = time.time()
        model.fit(X_train, y_train)
        end = time.time()
        times.append(end - start)
        pred = model.predict(X_valid)
        # print(confusion_matrix(y_valid, pred))
        acc = accuracy_score(pred, y_valid)
        accuracies.append(acc)
        
    print('====\nFinal results: \nMean accuracy:', np.mean(accuracies))
    results.append(np.mean(accuracies))
    tempos.append(np.mean(times))
    print('Mean traning model time: ', np.mean(times))

('\n\n======= Model: ', MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(16, 8), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=5, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False), '=======')
('====\nFinal results: \nMean accuracy:', 0.9466666666666667)
('Mean traning model time: ', 0.1121152400970459)
('\n\n======= Model: ', Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=5, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False), '=======')
('====\nFinal results: \nMean accuracy:', 0.8)
('Mean traning model time: '

In [16]:
data = {'Accuracy': results, 'Time': tempos, 'Model': models}
dataframe = pd.DataFrame(data=data)
dataframe = dataframe.sort_values(by=['Accuracy'], ascending=False)
dataframe

,Accuracy,Model,Time
3,0.953333,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',\n max_iter=-1, ...",0.002921
6,0.953333,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',\n max_iter=-1, ...",0.001636
0,0.946667,"MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',\n beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,\n ...",0.112115
2,0.913333,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',\n max_iter=-1,...",0.004117
5,0.913333,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovo', degree=3, gamma='scale', kernel='poly',\n max_iter=-1,...",0.002222
4,0.873333,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',\n max_iter=...",0.001877
7,0.873333,"SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,\n decision_function_shape='ovo', degree=3, gamma='scale', kernel='sigmoid',\n max_iter=...",0.001624
1,0.800000,"Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,\n fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_chang...",0.005045


In [17]:
dataframe.to_csv('iris_results.csv',index=False)